In [ ]:
# dependencies
!pip install huggingface_hub
!pip install timm

from google.colab import drive
from huggingface_hub import notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import re
import torch
import timm

drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [ ]:
# load from hugging face
#notebook_login()

#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

# save the model !!
#path = '/content/drive/My Drive/MyModels/mistral/Mixtral-7B-v0.1'
#model.save_pretrained(path)

In [ ]:
# load the model and tokenizer from drive
from transformers import CanineForMultipleChoice
model_choice = 'canine'
#model_choice = 'mistral'
#model_choice = 'mistral_instruct'

tokenizer = None
model = None

if model_choice == 'mistral':
    path_model = '/content/drive/My Drive/C5470prj/Mistral-7B-v0.1' #mistral simple
if model_choice == 'mistral_instruct':
    path_model = '/content/drive/My Drive/C5470prj/Mistral-7B-Instruct-v0.2' #mistral + instruct prompting
if model_choice == 'canine':
    path_model = "google/canine-s"

tokenizer = AutoTokenizer.from_pretrained(path_model)
model = AutoModelForCausalLM.from_pretrained(path_model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# for Mistral testing only
if 'Mistral' in model_choice:
    text = '''[INST] ### Instructions:
Below is an instruction that describes a
multiple choice task. Answer the following
multiple choice question by giving the
most appropriate response. Answer should
be one among options provided after the
question. Select the most suitable answer
while making the necessary assumptions.
Give only answer and a short explanation
of two or three sentences. Nothing else.

### Input:
Question: {A man shaves everyday, yet keeps his beard long.}
{a. He is a barber.}
{b. He wants to maintain his appearance.}
{c. He wants his girlfriend to buy him a razor.}
{d. None of above.}

### Output Format:
The correct answer is: {label} {explanation}
Change {label} to one of a, b, c or d in your output.
Change {explanation} to a short explanation.
Do not output anything else after it.  [/INST]

The correct answer is: '''

    inputs = tokenizer(text, return_tensors="pt")
# using Canine tokenizer:
#inputs = tokenizer(text, padding="longest", truncation=True, return_tensors="pt", return_token_type_ids=False)
# do this for different tokenizers!
#model.resize_token_embeddings(len(tokenizer))
    outputs = model.generate(**inputs, max_new_tokens=(50))

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] ### Instructions:
Below is an instruction that describes a
multiple choice task. Answer the following
multiple choice question by giving the
most appropriate response. Answer should
be one among options provided after the
question. Select the most suitable answer
while making the necessary assumptions.
Give only answer and a short explanation
of two or three sentences. Nothing else.

### Input:
Question: {A man shaves everyday, yet keeps his beard long.}
{a. He is a barber.}
{b. He wants to maintain his appearance.}
{c. He wants his girlfriend to buy him a razor.}
{d. None of above.}

Hint: The man can shave other people's beard.

### Output Format:
The correct answer is: {label} {explanation}
Change {label} to one of a, b, c or d in your output.
Change {explanation} to a short explanation.
Do not output anything else after it.  [/INST]

The correct answer is:  a. He is a barber.
Explanation: The man shaves everyday but keeps his own beard long, which is consistent with being a 

In [ ]:
import random
# Load the data
SP_train = np.load('/content/drive/My Drive/C5470prj/BrainTeaser/data/SP-train.npy', allow_pickle=True)
WP_train = np.load('/content/drive/My Drive/C5470prj/BrainTeaser/data/WP-train.npy', allow_pickle=True)

SP_train = SP_train[:49]  # if recover from crashes
WP_train = WP_train[:49]

# Function to format the input text according to the model's required format
def format_question(data):
    id = data['id']
    question = data['question']
    print("processing data id: " + id + " length: " + str(len(question)))

    choices = data['choice_list']
    choice_order = data['choice_order']
    formatted_choices = [f"{chr(97 + i)}. {choices[idx]}" for i, idx in enumerate(choice_order)]
    text = f"""###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. Answer the following multiple choice question by giving the most appropriate response. Answer should be one among options provided after the question. Select the most suitable answer while making the necessary assumptions. Give only answer and a short explanation of two or three sentences. Nothing else.

    ### Input:
    Question: {question}
    {formatted_choices[0]}
    {formatted_choices[1]}
    {formatted_choices[2]}
    {formatted_choices[3]}

    ### Output Format:
    The correct answer is: {{label}}{{explanation}}
    Change {{label}} to one of a, b, c or d in your output.
    Change {{explanation}} to a short explanation.
    Do not output anything else after it. [/INST]

    The correct answer is: """
    return text, id

# stats
correct_answers = 0
question_count = 0
SP_count = 0
SP_correct = 0
WP_count = 0
WP_correct = 0
wp = False
sp = False

# do this for different tokenizers!
#model.resize_token_embeddings(len(tokenizer))

for data in np.concatenate((SP_train, WP_train)):
    wp = False
    sp = False

    formatted_text, id = format_question(data)
    if '_' in id: # exclude reconstructed questions during eval
        continue
    if "WP" in id:
        wp = True
    if "SP" in id:
        sp = True

    question_count += 1
    if wp:
      WP_count += 1
    if sp:
      SP_count += 1

    # using mistral tokenizer:
    inputs = tokenizer(formatted_text, return_tensors="pt")
    # using Canine tokenizer:
    #inputs = tokenizer(formatted_text, padding="longest", truncation=True, return_tensors="pt", return_token_type_ids=False)

    # inference
    outputs = model.generate(**inputs, max_new_tokens=20) # 6 for base, 30 for instruct
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)

    match_str = "The correct answer is:"
    predicted_answer = ''
    match = re.search(r'[^a-zA-Z]([a-d])[^a-zA-Z]', response[len(formatted_text):])
    if match:
        predicted_answer = match.group(1)
    else:
        predicted_answer = chr(97 + random.randint(0, 2))

    # Find the index of the correct answer in the choice order
    correct_label = ''
    # sanitize dataset
    if data['answer'] not in data['choice_list']:
        question_count -= 1
        if wp:
            WP_count -= 1
        if sp:
            SP_count -= 1
        continue

    correct_index = data['choice_list'].index(data['answer'])
    for i, ordered_i in enumerate(data['choice_order']):
        if ordered_i == correct_index:
            correct_label = chr(97 + i)

    if predicted_answer == correct_label:
        correct_answers += 1
        if wp:
            WP_correct += 1
        if sp:
            SP_correct += 1
    print("evaluated data id: " + id + ", predicted: " + predicted_answer + ", correct: " + correct_label + " total acc: " + str(correct_answers / question_count))
    if SP_count != 0:
        print(" SP acc: " + str(SP_correct / SP_count) + " correct: " + str(SP_correct) + "  count: "  + str(SP_count))
    if WP_count != 0:
        print(" WP acc: " + str(WP_correct / WP_count) + " correct: " + str(WP_correct) + "  count: "  + str(WP_count))

# Calculate accuracy
accuracy = correct_answers / question_count
print(f"Total Accuracy: {accuracy:.2f}")



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


processing data id: SP-0 length: 139


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. Answer the following multiple choice question by giving the most appropriate response. Answer should be one among options provided after the question. Select the most suitable answer while making the necessary assumptions. Give only answer and a short explanation of two or three sentences. Nothing else.

    ### Input:
    Question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?
    a. Each daughter shares the same brother.
    b. Some daughters get married and have their own family.
    c. Some brothers were not loved by family and moved away.
    d. None of above.

    ### Output Format:
    The correct answer is: {label}{explanation}
    Change {label} to one of a, b, c or d in your output.
    Change {explanation} to a short explanation.
    Do not output anything else after it. [/INST]

    The corr

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. Answer the following multiple choice question by giving the most appropriate response. Answer should be one among options provided after the question. Select the most suitable answer while making the necessary assumptions. Give only answer and a short explanation of two or three sentences. Nothing else.

    ### Input:
    Question: Which word in the English language becomes shorter when it is lengthened?
    a. Short.
    b. Truncation.
    c. Abbreviation.
    d. None of above.

    ### Output Format:
    The correct answer is: {label}{explanation}
    Change {label} to one of a, b, c or d in your output.
    Change {explanation} to a short explanation.
    Do not output anything else after it. [/INST]

    The correct answer is:  c. Abbreviation.
    An abbreviation is a short form of a
evaluated data id: WP-14, predicted: c, correct: a total acc: 0.25806451612903225
 SP acc: 0.29411764705882

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. Answer the following multiple choice question by giving the most appropriate response. Answer should be one among options provided after the question. Select the most suitable answer while making the necessary assumptions. Give only answer and a short explanation of two or three sentences. Nothing else.

    ### Input:
    Question: What dress does everyone have, but no one wears?
    a. Address.
    b. Suit.
    c. Wedding dress.
    d. None of above.

    ### Output Format:
    The correct answer is: {label}{explanation}
    Change {label} to one of a, b, c or d in your output.
    Change {explanation} to a short explanation.
    Do not output anything else after it. [/INST]

    The correct answer is:  d. None of above.
    There is no dress that everyone has but no one wears
evaluated data id: WP-16, predicted: d, correct: a total acc: 0.25
 SP acc: 0.29411764705882354 correct: 5  count: 17


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. Answer the following multiple choice question by giving the most appropriate response. Answer should be one among options provided after the question. Select the most suitable answer while making the necessary assumptions. Give only answer and a short explanation of two or three sentences. Nothing else.

    ### Input:
    Question: What letters of the alphabet come too late for supper?
    a. Those that come after T.
    b. Those that at the end of alphabet.
    c. Those that at the middle of alphabet.
    d. None of above.

    ### Output Format:
    The correct answer is: {label}{explanation}
    Change {label} to one of a, b, c or d in your output.
    Change {explanation} to a short explanation.
    Do not output anything else after it. [/INST]

    The correct answer is:  a. Those that come after T.
    Explanation: The question asks for letters that
evaluated data id: WP-17, predicted: a,

In [ ]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
choice0 = "It is eaten with a fork and a knife."
choice1 = "It is eaten while held in the hand."
labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1

encoding = tokenizer([prompt, prompt], [choice0, choice1], return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=labels)  # batch size is 1

# the linear classifier still needs to be trained
loss = outputs.loss
logits = outputs.logits

##REsults
evaluated data id: WP-18, predicted: c, correct: a total acc: 0.2647058823529412
 SP acc: 0.29411764705882354 correct: 5  count: 17
 WP acc: 0.23529411764705882 correct: 4  count: 17
Total Accuracy: 0.26


evaluated data id: SP-208, predicted: a, correct: a total acc: 0.40236686390532544
 SP acc: 0.40236686390532544 correct: 68  count: 169

evaluated data id: WP-163, predicted: b, correct: a total acc: 0.3969465648854962
 WP acc: 0.3969465648854962 correct: 52  count: 131
Total Accuracy: 0.40



